# LLaVA-NEXT Env:
python3.10下pip安装

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
!cd LLaVA
%pip install --upgrade pip  # enable PEP 660 support
%pip install -e .

## Run Code:

In [2]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

model_path = "liuhaotian/llava-v1.6-mistral-7b" # 因内存限制，运行了7b参数模型，如果有条件可尝试："liuhaotian/llava-v1.6-34b" 
model_name = get_model_name_from_path(model_path)

In [4]:
import torch

class Configs:
    # model_path = "liuhaotian/llava-v1.6-mistral-7b" url
    model_path = "./llava-v1.6-mistral-7b" #本地模型参数
    model_base = None
    model_name = model_name
    image_file = ""
    query = ""
    conv_mode = None
    sep = ","
    temperature = 0.2
    top_p = None
    num_beams = 1
    max_new_tokens = 512
    device = "cuda"

import requests
from PIL import Image
from io import BytesIO
import re

def image_parser(args):
    out = args.image_file.split(args.sep)
    return out

def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image


def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out

def get_device_map() -> str:
    return 'cuda' if torch.cuda.is_available() else 'cpu'

dev = get_device_map()  # 'cpu'
print(dev)
args = Configs()

cuda


In [5]:
test = "./llava-v1.6-mistral-7b/abc.png"

In [7]:
from warnings import filterwarnings
import re
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)


filterwarnings("ignore")
tokenizer, model, image_processor, context_len = load_pretrained_model(
        args.model_path, args.model_base, model_name, load_4bit=True
    )

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.29s/it]


mistral done.


### Multi Page

In [8]:
import os
import json

# imgs_dir
loaded_data = os.listdir("data/docVQA-M")
loaded_data.sort()
loaded_data = loaded_data[:-1]

print(loaded_data)
# caculate the number of multipage imgs
num = 0
dummy_data = ""
img_list = []
for data in loaded_data:
    if num == 50:
        break
    data = data.split('_')[0]
    if dummy_data != data:
        dummy_data = data
        img_list.append(data)
        num +=1
#print(img_list)

gt_json = json.load(open("data/docVQA-M/qas/train.json","r+"))['data']
filtered_gt = {}
for key in img_list:
    filtered_gt[key] = []

for gt in gt_json:
    if gt["doc_id"] in img_list:
        filtered_gt[gt["doc_id"]].append(gt)

print(filtered_gt)
print(len(filtered_gt))

['ffbf0023_p0.jpg', 'ffbf0023_p1.jpg', 'ffbf0023_p2.jpg', 'ffbf0023_p3.jpg', 'ffbf0023_p4.jpg', 'ffbf0023_p5.jpg', 'ffbf0023_p6.jpg', 'ffbf0023_p7.jpg', 'ffbf0227_p0.jpg', 'ffbf0227_p1.jpg', 'ffbf0227_p2.jpg', 'ffbg0227_p0.jpg', 'ffbg0227_p1.jpg', 'ffbg0227_p2.jpg', 'ffbg0227_p3.jpg', 'ffbg0227_p4.jpg', 'ffbg0227_p5.jpg', 'ffbg0227_p6.jpg', 'ffbl0226_p0.jpg', 'ffbl0226_p1.jpg', 'ffbl0226_p2.jpg', 'ffbx0227_p0.jpg', 'ffbx0227_p1.jpg', 'ffbx0227_p2.jpg', 'ffbx0227_p3.jpg', 'ffbx0227_p4.jpg', 'ffbx0227_p5.jpg', 'ffbx0227_p6.jpg', 'ffbx0227_p7.jpg', 'ffcc0228_p0.jpg', 'ffcn0226_p0.jpg', 'ffcn0226_p1.jpg', 'ffcn0226_p10.jpg', 'ffcn0226_p11.jpg', 'ffcn0226_p12.jpg', 'ffcn0226_p13.jpg', 'ffcn0226_p14.jpg', 'ffcn0226_p15.jpg', 'ffcn0226_p16.jpg', 'ffcn0226_p17.jpg', 'ffcn0226_p18.jpg', 'ffcn0226_p19.jpg', 'ffcn0226_p2.jpg', 'ffcn0226_p20.jpg', 'ffcn0226_p21.jpg', 'ffcn0226_p22.jpg', 'ffcn0226_p23.jpg', 'ffcn0226_p24.jpg', 'ffcn0226_p25.jpg', 'ffcn0226_p26.jpg', 'ffcn0226_p27.jpg', 'ffcn0226_p2

In [9]:
from llava.conversation import conv_templates, SeparatorStyle
from llava.mm_utils import process_images, tokenizer_image_token
import PIL.Image as Image
import pandas as pd

df = pd.DataFrame(columns=['image_file', 'question', 'GT', "prediction"])
# loop for muti-pages
for key, gt in filtered_gt.items():
    print("-------------------------------------------- Muti page: ", key, "----------------------------------------------------------------")
    for question in gt:
        img_file = []
        page_id = question['answer_page_idx']
        for page in question["page_ids"]:
            img_file.append(os.path.join("data/docVQA-M", page +".jpg"))
        img_file = ','.join(img_file)
        print(img_file)
        #img_file = os.path.join("data/docVQA-M", question["page_ids"][int(page_id)]+ '.jpg')
        query = "Return the answer only. Do not return any output descriptions or explanations, only the answer. "+ question["question"]
        answer = question["answers"]
    
    # model input
        qs = query
        image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
        if IMAGE_PLACEHOLDER in qs:
            if model.config.mm_use_im_start_end:
                qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
            else:
                qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
        else:
            if model.config.mm_use_im_start_end:
                qs = image_token_se + "\n" + qs
            else:
                qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

        if "llama-2" in model_name.lower():
            conv_mode = "llava_llama_2"
        elif "mistral" in model_name.lower():
            conv_mode = "mistral_instruct"
        elif "v1.6-34b" in model_name.lower():
            conv_mode = "chatml_direct"
        elif "v1" in model_name.lower():
            conv_mode = "llava_v1"
        elif "mpt" in model_name.lower():
            conv_mode = "mpt"
        else:
            conv_mode = "llava_v0"

        if args.conv_mode is not None and conv_mode != args.conv_mode:
            print(
                "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
                    conv_mode, args.conv_mode, args.conv_mode
                )
            )
        else:
            args.conv_mode = conv_mode

        args.image_file =  img_file
        conv = conv_templates[args.conv_mode].copy()
        conv.append_message(conv.roles[0], qs)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()
        image_files = image_parser(args)
        images = load_images(image_files)
        image_sizes = [x.size for x in images]
        images_tensor = process_images(
            images,
            image_processor,
            model.config
        )
        if not isinstance(images_tensor,torch.Tensor):
            continue
        images_tensor= images_tensor.to(model.device, dtype=torch.float16)
        # print(images_tensor.size())

        input_ids = (
            tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
            .unsqueeze(0)
            .cuda()
        )

        with torch.inference_mode():
            output_ids = model.generate(
                input_ids,
                images=images_tensor,
                image_sizes=image_sizes,
                do_sample=True if args.temperature > 0 else False,
                temperature=args.temperature,
                top_p=args.top_p,
                num_beams=args.num_beams,
                max_new_tokens=args.max_new_tokens,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id
            )

        outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
        out_dict = {"image_file":img_file, "question": qs, "GT":question["answers"], "prediction": outputs}
        df.loc[len(df)] = out_dict
        print(out_dict)

-------------------------------------------- Muti page:  ffbf0023 ----------------------------------------------------------------
data/docVQA-M/ffbf0023_p0.jpg,data/docVQA-M/ffbf0023_p1.jpg,data/docVQA-M/ffbf0023_p2.jpg,data/docVQA-M/ffbf0023_p3.jpg,data/docVQA-M/ffbf0023_p4.jpg,data/docVQA-M/ffbf0023_p5.jpg,data/docVQA-M/ffbf0023_p6.jpg,data/docVQA-M/ffbf0023_p7.jpg
{'image_file': 'data/docVQA-M/ffbf0023_p0.jpg,data/docVQA-M/ffbf0023_p1.jpg,data/docVQA-M/ffbf0023_p2.jpg,data/docVQA-M/ffbf0023_p3.jpg,data/docVQA-M/ffbf0023_p4.jpg,data/docVQA-M/ffbf0023_p5.jpg,data/docVQA-M/ffbf0023_p6.jpg,data/docVQA-M/ffbf0023_p7.jpg', 'question': '<image>\nReturn the answer only. Do not return any output descriptions or explanations, only the answer. When is the contract effective date?', 'GT': ['7 - 1 - 99'], 'prediction': '1/1/2002'}
data/docVQA-M/ffbf0023_p0.jpg,data/docVQA-M/ffbf0023_p1.jpg,data/docVQA-M/ffbf0023_p2.jpg,data/docVQA-M/ffbf0023_p3.jpg,data/docVQA-M/ffbf0023_p4.jpg,data/docVQA-M/ff

In [13]:
# df.to_excel("docVQA-Multi-1-img-output.xlsx",index=False)
df.to_excel("docVQA-Multi-all-output.xlsx",index=False)